# Professional Crypto Market Analytics

This notebook delivers a comprehensive, institutional-grade analysis of the top cryptocurrencies, including Sharpe ratios, price vs. all-time high, dominance metrics, functional groupings, and advanced visualizations. All code is modular and ready for due diligence.

## Table of Contents
1. Introduction & Methodology
2. Data Import & Ticker Groupings
3. Risk/Return Analysis (Sharpe Ratios)
4. Price vs. All-Time High (ATH)
5. Dominance Metrics (BTC, ETH, etc.)
6. Functional Group Analysis (Layer 1s, DeFi, AI, etc.)
7. Advanced Visualizations
8. Insights & Recommendations

## 1. Introduction & Methodology

This notebook analyzes the top cryptocurrencies by market cap and function, using professional risk/return metrics, dominance calculations, and group-based analytics. All code is modular and ready for institutional review.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
plt.style.use('dark_background')

risk_free_rate = 0.045  # US 10-year Treasury yield, 2025

## 2. Data Import & Ticker Groupings

Define tickers and groupings for Layer 1s, DeFi, AI, NFTs, and more.

In [ ]:
crypto_tickers = {
    'Bitcoin': 'BTC-USD',
    'Ethereum': 'ETH-USD',
    'Solana': 'SOL-USD',
    'Ripple': 'XRP-USD',
    'Hedera': 'HBAR-USD',
    'Near Protocol': 'NEAR-USD',
    'Cardano': 'ADA-USD',
    'Avalanche': 'AVAX-USD',
    'Polkadot': 'DOT-USD',
    'Sui': 'SUI20947-USD',
    'Chainlink': 'LINK-USD',
    'Uniswap': 'UNI-USD',
    'Aave': 'AAVE-USD',
    'Bittensor': 'TAO22974-USD',
    'Raydium': 'RAY-USD',
    'Morpho': 'MORPHO-USD',
    'Aerodrome': 'AERO-USD',
    'Dogecoin': 'DOGE-USD',
    'Bonk': 'BONK-USD',
    'Pepe': 'PEPE-USD',
    'Orca': 'ORCA-USD',
    'Curve': 'CRV-USD',
    'Drift': 'DRIFT-USD',
    'Helium': 'HNT-USD',
    'Fetch.ai': 'FET-USD',
    'Grass': 'GRASS32956-USD',
}

crypto_groups = {
    'Layer 1s': ['BTC-USD', 'ETH-USD', 'SOL-USD', 'AVAX-USD', 'DOT-USD', 'ADA-USD', 'NEAR-USD', 'SUI20947-USD', 'HBAR-USD'],
    'DeFi': ['UNI-USD', 'AAVE-USD', 'RAY-USD', 'MORPHO-USD', 'AERO-USD', 'CRV-USD', 'ORCA-USD', 'DRIFT-USD'],
    'AI': ['FET-USD', 'TAO22974-USD', 'GRASS32956-USD'],
    'NFTs/Metaverse': ['BONK-USD', 'PEPE-USD', 'DOGE-USD'],
    'Oracles': ['LINK-USD'],
    'Other': ['HNT-USD'],
}

start_date = '2021-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')
prices = yf.download(list(crypto_tickers.values()), start=start_date, end=end_date)['Close']
prices = prices.ffill().dropna(how='all')

## 3. Risk/Return Analysis (Sharpe Ratios)

Calculate annualized return, volatility, and Sharpe ratio for each crypto. Sort by Sharpe ratio.

In [ ]:
returns = prices.pct_change().dropna()
ann_return = returns.mean() * 252
ann_vol = returns.std() * np.sqrt(252)
sharpe = (ann_return - risk_free_rate) / ann_vol
sharpe_table = pd.DataFrame({'Ann. Return': ann_return, 'Ann. Volatility': ann_vol, 'Sharpe Ratio': sharpe})
sharpe_table = sharpe_table.sort_values('Sharpe Ratio', ascending=False)
display(sharpe_table.style.format('{:.2%}', subset=['Ann. Return', 'Ann. Volatility']).format('{:.2f}', subset=['Sharpe Ratio']))

## 4. Price vs. All-Time High (ATH)

Calculate and plot the current price as a percentage of its all-time high for each crypto.

In [ ]:
ath = prices.max()
current = prices.iloc[-1]
ath_pct = (current / ath * 100).sort_values(ascending=False)
ath_table = pd.DataFrame({'Current Price': current, 'ATH': ath, 'Current/ATH (%)': ath_pct})
display(ath_table.sort_values('Current/ATH (%)', ascending=False))
plt.figure(figsize=(12,6))
ath_pct.plot(kind='bar', color='skyblue')
plt.title('Current Price as % of All-Time High')
plt.ylabel('% of ATH')
plt.tight_layout()
plt.show()

## 5. Dominance Metrics (BTC, ETH, etc.)

Visualize Bitcoin dominance, Ethereum dominance (ex-Bitcoin), and group dominance over time.

In [ ]:
# Calculate market cap proxies (using price only, for demo)
market_caps = prices * 1  # Replace with real market cap data if available
btc_dominance = prices['BTC-USD'] / prices.sum(axis=1)
eth_dominance = prices['ETH-USD'] / (prices.sum(axis=1) - prices['BTC-USD'])
plt.figure(figsize=(14,6))
btc_dominance.plot(label='BTC Dominance')
eth_dominance.plot(label='ETH Dominance ex-BTC')
plt.title('Bitcoin & Ethereum Dominance')
plt.ylabel('Dominance (%)')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Functional Group Analysis

Aggregate and plot performance by group (Layer 1s, DeFi, AI, etc.).

In [ ]:
# Group performance (equal-weighted)
group_perf = {}
for group, tickers in crypto_groups.items():
    valid = [t for t in tickers if t in prices.columns]
    if valid:
        group_perf[group] = (prices[valid].mean(axis=1) / prices[valid].mean(axis=1).iloc[0])
pd.DataFrame(group_perf).plot(figsize=(14,7))
plt.title('Group Performance (Normalized)')
plt.ylabel('Growth (since 2021)')
plt.tight_layout()
plt.show()

## 7. Advanced Visualizations

Rolling Sharpe ratios, volatility, and drawdowns for top cryptos.

In [ ]:
# Rolling Sharpe ratio (90-day)
rolling_sharpe = (returns.rolling(90).mean() - risk_free_rate/252) / returns.rolling(90).std() * np.sqrt(252)
plt.figure(figsize=(14,7))
for t in ['BTC-USD', 'ETH-USD', 'SOL-USD', 'AVAX-USD']:
    if t in rolling_sharpe.columns:
        rolling_sharpe[t].plot(label=t)
plt.title('90-Day Rolling Sharpe Ratio')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Drawdown plot for top cryptos
def drawdown(prices):
    roll_max = prices.cummax()
    return prices / roll_max - 1
plt.figure(figsize=(14,7))
for t in ['BTC-USD', 'ETH-USD', 'SOL-USD', 'AVAX-USD']:
    if t in prices.columns:
        dd = drawdown(prices[t])
        dd.plot(label=t)
plt.title('Drawdown (from ATH)')
plt.ylabel('Drawdown')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Insights & Recommendations

- Sharpe ratios highlight which cryptos have delivered the best risk-adjusted returns recently.
- Most cryptos are well below their all-time highs, indicating potential for mean reversion or continued risk.
- Bitcoin remains dominant, but Layer 1s and AI coins are gaining traction.
- Diversify across functions, but size positions based on volatility and drawdown risk.
- Use rolling Sharpe and drawdown analytics to inform tactical allocation and risk management.